# Data Exploration

### T10-4
As a USER, I want to have a Data Exploration Section in the notebook so that I can easily locate the upload and playback functions in the notebook.

# Inference

### T10-7
As a USER, I want to have an Inference Section in the notebook so that I can easily locate the Inference Section in the notebook.

# Training

### T10-6
As a USER, I want to have a Training Section in the notebook so that I can easily locate the training related functions in the notebook.